In [1]:
%reset -f
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re, sqlite3, pickle, time, datetime, random, os
from dateutil.relativedelta import relativedelta

pd.options.display.float_format = '{:,.4f}'.format

# Load Processed parameters and data

In [2]:
##############################################################################
#######################################
### Relevant fields for pre-processing
#######################################
reference_date = datetime.datetime(1970,1,1)

##### Fields in table patients ##########
patient_field         = 'patient_id'
age_field             = 'age_at_admission\r'
sex_field             = 'sex'
ethnic_field          = 'ethnic_origin'
death_ind_field       = 'death_indicator'
death_date_field      = 'date_of_death'
death_date_code_field = 'date_code_of_death'
mort_in_hosp_field    = 'Thirty_day_mort'

##### Fields in table admissions ########
admission_field           = 'admission_id'
diagnosis_field           = 'episode_diagnoses'
admn_date_field           = 'admission_date_time'
admn_discharge_field      = 'discharge_date_time'
admn_date_code_field      = 'admission_date_code_time'
admn_discharge_code_field = 'discharge_date_code_time'
lengthofstay_field        = 'lengthofstay'
isPneumonia_field         = 'isPneumonia'
mortal_admin_field        = 'mortal_admin'
comorbidity_field         = 'Comorbidity_score'
icu_admin_field           = 'icu_count\r'
no_eobs_field             = 'no_obs_eobs'
no_haematology_field      = 'no_haematology_eobs'
meds_drug_field           = 'Drug_supplied'
specific_comorb_field     = 'Specific Comorbidity'
prev_admin_field          = 'had_Prev_admin'
has_spin_field            = 'adm_has_spin'
cubr65_field              = 'CURB65'
antibiot_4h_field         = 'antibiotic_4h'

##### Fields in table eObservations #####
eObs_time_field      = 'timestamp'
eObs_time_code_field = 'timestamp_code'
eObs_time_prev_obs   = 'time_since_prev_obs_in_mins'
eObs_resprate_field  = 'rr'
eObs_sbp_field       = 'sbp'
eObs_dbp_field       = 'dbp'
eObs_newscore_field  = 'ews'
eObs_heartrate_field = 'heart_rate'
eObs_temptr_field    = 'temperature\r'
eObs_oxygen_field    = 'Oxygen_Saturation'

##### Fields in table haematological tests #####
test_code_field      = 'local_test_code'
test_time_field      = 'sample_collection_date_time'
test_time_code_field = 'sample_collection_date_code_time'

In [3]:
data = pickle.load( open('DataFrame_pickles/df_patients_admissions_2019_20_v4.pickle', 'rb'))
df_patients   = data[0]
df_admissions = data[1]

df_eobs = pickle.load( open("DataFrame_pickles/df_eobs_3d.pickle", "rb" ))
df_haem = pickle.load( open('DataFrame_pickles/new_haematology_3d.pickle', 'rb')) 
df_haem = df_haem.rename(columns = {'sample_collection_date_time':'timestamp'})

# Selection of Blood Tests


In [4]:
df = pd.DataFrame(df_haem.count())
df = df.rename(columns = {0: 'Count'}).sort_values(by = ['Count'], ascending = False)
df['Per'] = df['Count'] / len(df_haem)
display(df)
BT_include = df[df['Per'] >= 0.7].index.tolist()[2:]
BT_include = ['CREA','UREA','K','GFR','WBC','PLT','HCT','HGB','RBC','MCH','MCV','NEUAB','TLYMAB',
              'EOSAB','MONAB','BASAB','ALB','ALP','BILI']

print("------------------------------------------------------")
print("Number of Blood Test to include in the merging dataset", len(BT_include))
print("")
print(BT_include)

,Count,Per
admission_id,99096,1.0000
HGB,99096,1.0000
CREA,99096,1.0000
UREA,99096,1.0000
K,99096,1.0000
WBC,99096,1.0000
timestamp,99096,1.0000
MCV,99084,0.9999
MCH,99084,0.9999
RBC,99084,0.9999


------------------------------------------------------
Number of Blood Test to include in the merging dataset 19

['CREA', 'UREA', 'K', 'GFR', 'WBC', 'PLT', 'HCT', 'HGB', 'RBC', 'MCH', 'MCV', 'NEUAB', 'TLYMAB', 'EOSAB', 'MONAB', 'BASAB', 'ALB', 'ALP', 'BILI']


In [5]:
df_haem_merge = df_haem[[admission_field, eObs_time_field] + BT_include]

In [6]:
display(df_haem_merge.describe())
display(df_eobs.describe())

,admission_id,timestamp,CREA,UREA,K,GFR,WBC,PLT,HCT,HGB,...,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI
count,99096,99096,"99,096.0000","99,096.0000","99,096.0000","99,012.0000","99,096.0000","99,024.0000","99,084.0000","99,096.0000",...,"99,084.0000","99,084.0000","98,076.0000","98,064.0000","98,064.0000","98,040.0000","98,028.0000","97,668.0000","97,656.0000","97,068.0000"
unique,8258,2813,"12,603.0000","16,265.0000","4,847.0000","8,182.0000","16,854.0000","20,028.0000","18,412.0000","8,219.0000",...,"12,708.0000","4,519.0000","38,239.0000","18,190.0000","7,902.0000","11,689.0000","3,255.0000","3,915.0000","11,136.0000","5,018.0000"
top,1671170,2021-01-19 12:00:00,330.0000,30.0000,4.2000,90.0015,3.0000,90.0000,0.3650,124.0000,...,29.5000,90.0000,15.0000,0.3000,0.0100,1.2000,0.0200,38.0000,200.0000,6.0000
freq,12,85,"2,731.0000","1,790.0000","2,802.0000","28,747.0000","1,804.0000","4,032.0000",264.0000,753.0000,...,697.0000,"2,676.0000","6,168.0000","1,546.0000","7,356.0000","2,561.0000","10,014.0000","3,546.0000","6,386.0000","4,389.0000"


,Oxygen_Saturation,Assisted_O2
count,"3,219,264.0000","3,219,264.0000"
mean,95.5850,0.4990
std,2.4089,0.5000
min,60.0000,0.0000
25%,94.5000,0.0000
50%,96.0000,0.0000
75%,97.0000,1.0000
max,100.0000,1.0000


In [7]:
A = df_admissions[admission_field].unique().tolist()
B = df_eobs[admission_field].unique().tolist()
C = df_haem_merge[admission_field].unique().tolist()
print("No. admissions in admissions", len(A))
print("No. admissions in eobs", len(B))
print("No. admissions in haematology", len(C))

D = set(A)&set(B)&set(C)

print("No. admissions in all", len(D))
admissions_to_process = list(D)

No. admissions in admissions 14714
No. admissions in eobs 11178
No. admissions in haematology 8258
No. admissions in all 8258


In [8]:
df = pd.merge(df_eobs[df_eobs[admission_field].isin(admissions_to_process)], df_haem_merge, on = [admission_field, eObs_time_field],how="left")

In [9]:
df.describe()

,Oxygen_Saturation,Assisted_O2
count,"2,378,304.0000","2,378,304.0000"
mean,95.6396,0.5024
std,2.4119,0.5000
min,60.0000,0.0000
25%,94.5882,0.0000
50%,96.0000,1.0000
75%,97.0000,1.0000
max,100.0000,1.0000


### Merging check Point before interpolation

In [10]:
admin = 36937728
admin = 54960131

print("__________________________________________________________________________________________________________")
print("ADMISSION INFORMATION")
display(df_admissions[df_admissions[admission_field]==admin])

print("")
print("")
print("=========================== time series EOBS table ====================================")
display(df_eobs[df_eobs[admission_field] == admin].iloc[:4])
print("")
print("")
print("=========================== time series HAEMATOLOGY info table ====================================")
display(df_haem_merge[df_haem_merge[admission_field] == admin].iloc[:4])
print("")
print("")
print("=========================== time series HMIXED table ====================================")
display(df[df[admission_field] == admin].iloc[:4])

print("__________________________________________________________________________________________________________")
print("")
print("df_eobs, number of records:", len(df_eobs[df_eobs[admission_field] == admin]))

print("df_haem, number of records:", len(df_haem_merge[df_haem_merge[admission_field] == admin]))

print("df_mixed number of records", len(df[df[admission_field] == admin]))


__________________________________________________________________________________________________________
ADMISSION INFORMATION


,admission_id,episode_diagnoses,patient_id,age_at_admission\r,admission_date_time,admission_date_code_time,discharge_date_time,discharge_date_code_time,lengthofstay,isPneumonia,...,icu_count\r,Comorbidity_score,no_obs_eobs,no_haematology_eobs,receivedMedicines,Specific Comorbidity,had_Prev_admin,adm_has_spin,CURB65,antibiotic_4h
19005,54960131,"||C18.4,C77.2,Z50.1,J18.1,M13.9,F32.9,F41.9,D6...",17278697,58,2020-03-19 07:15:00,"18,340.3021",2020-03-28 19:00:00,"18,349.7917",9 days 11:45:00,1,...,0,6,53,8,0,0,1.0000,NaN,NaN,NaN




=========================== time series EOBS table ====================================


,admission_id,timestamp,time_since_prev_obs_in_mins,rr,ews,heart_rate,temperature\r,timestamp_code,sbp,dbp,Oxygen_Saturation,Assisted_O2
0,54960131,2020-03-19 07:45:00,15,16.0000,1,77.0000,36.6000,"18,340.3274",109.0000,78.0000,98.0000,0.0000
1,54960131,2020-03-19 08:00:00,15,15.9900,1,76.7900,36.6030,"18,340.3377",108.9800,77.8600,98.0200,0.0000
2,54960131,2020-03-19 08:15:00,15,15.9800,1,76.5800,36.6060,"18,340.3481",108.9600,77.7200,98.0400,0.0000
3,54960131,2020-03-19 08:30:00,15,15.9700,1,76.3700,36.6090,"18,340.3584",108.9400,77.5800,98.0600,0.0000




=========================== time series HAEMATOLOGY info table ====================================


,admission_id,timestamp,CREA,UREA,K,GFR,WBC,PLT,HCT,HGB,...,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI
0,54960131,2020-03-19 12:00:00,61.0000,5.8000,3.8000,90.0015,7.7000,215.0000,0.3900,126.0000,...,28.6000,89.0000,6.6900,0.8300,0.0300,0.1000,0.0100,42.0000,108.0000,8.0000
1,54960131,2020-03-19 18:00:00,63.3333,5.5333,4.0667,87.3343,8.3000,217.0000,0.3733,121.0000,...,28.7667,89.0000,7.0567,0.9033,0.0267,0.3000,0.0067,40.3333,102.3333,8.0000
2,54960131,2020-03-20 00:00:00,65.6667,5.2667,4.3333,84.6672,8.9000,219.0000,0.3567,116.0000,...,28.9333,89.0000,7.4233,0.9767,0.0233,0.5000,0.0033,38.6667,96.6667,8.0000
3,54960131,2020-03-20 06:00:00,68.0000,5.0000,4.6000,82.0000,9.5000,221.0000,0.3400,111.0000,...,29.1000,89.0000,7.7900,1.0500,0.0200,0.7000,0.0000,37.0000,91.0000,8.0000




=========================== time series HMIXED table ====================================


,admission_id,timestamp,time_since_prev_obs_in_mins,rr,ews,heart_rate,temperature\r,timestamp_code,sbp,dbp,...,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI
1358208,54960131,2020-03-19 07:45:00,15,16.0000,1,77.0000,36.6000,"18,340.3274",109.0000,78.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1358209,54960131,2020-03-19 08:00:00,15,15.9900,1,76.7900,36.6030,"18,340.3377",108.9800,77.8600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1358210,54960131,2020-03-19 08:15:00,15,15.9800,1,76.5800,36.6060,"18,340.3481",108.9600,77.7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1358211,54960131,2020-03-19 08:30:00,15,15.9700,1,76.3700,36.6090,"18,340.3584",108.9400,77.5800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


__________________________________________________________________________________________________________

df_eobs, number of records: 288
df_haem, number of records: 12
df_mixed number of records 288


## Interpolation of Haematology Features

In [11]:
def interpolate_by_method(series, method, plot = 0):
    if ('polynomial' in method) or ('nearest' in method):
        interpolate = series.interpolate(method = method[:-1], order = int(method[-1]))
    else:
        interpolate = series.interpolate(method = method, order = method)
    if plot != 0: interpolate.plot()
    return interpolate

def fill_top_bottom_values(series):
    interp =series.copy()
    first_non_na = 0
    last_non_na  = 0
    for idx,val in interp.items():
        if (val != val) and (first_non_na == 0): # if val is nan and first_non_na = 0
            continue
        elif not (val != val) and (first_non_na == 0):
            first_non_na = idx       
        elif (val != val) and (first_non_na != 0) and (last_non_na == 0):
            last_non_na = idx - relativedelta(minutes = 15)
    
    if first_non_na !=0:
        for i in range(len(interp.loc[:first_non_na]) - 1):
            time_to_change  = first_non_na - relativedelta(minutes = 15 * (i+1))
            fifth_next_time = time_to_change + relativedelta(minutes = 15*10)
            interp.at[time_to_change] = interp.loc[time_to_change:fifth_next_time].mean()

    if last_non_na !=0:
        for i in range(len(interp.loc[last_non_na:])-1):
            time_to_change  = last_non_na + relativedelta(minutes = 15 * (i+1))
            fifth_prev_time = time_to_change - relativedelta(minutes = 15*10)
            interp.at[time_to_change] = interp.loc[fifth_prev_time:time_to_change].mean()
    return interp

In [12]:
t =time.time()
df_eobs_haem_mx = pd.DataFrame(columns = df.columns)
list_features = list(df_haem_merge.columns[2:])

for adm in df[admission_field].unique().tolist():

    df_eob_mx_adm   = df[df[admission_field] == adm].copy()
    upsamp_times    = df_eob_mx_adm[eObs_time_field].values
    
    for idx_feat, feature in enumerate(list_features):
        values  = df_eob_mx_adm[feature].values
        series_ = pd.Series(values, index=upsamp_times)
        if len(series_) - series_.isna().sum() < 4: continue
        interp  = interpolate_by_method(series_, 'linear')
        if len(interp) == interp.isna().sum(): continue
            
        interp = interp.fillna(method="bfill")    
        interp = interp.fillna(method="ffill")    
    
        #interp  = fill_top_bottom_values(interp)
        
        df_eob_mx_adm[feature] = interp.values
        
    df_eobs_haem_mx = pd.concat([df_eobs_haem_mx, df_eob_mx_adm])
   
print("elpased:", time.time()-t)

elpased: 5271.919864654541


In [13]:
# --------------------------------------------------------------------
#Reduce the time series to 144 samples in the three days. which means a reading every 30 minutes. 
# --------------------------------------------------------------------
df_eobs_haem_mx = df_eobs_haem_mx.iloc[::2]
df_eobs_haem_mx.reset_index(drop=True)

,admission_id,timestamp,time_since_prev_obs_in_mins,rr,ews,heart_rate,temperature\r,timestamp_code,sbp,dbp,...,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI
0,47438179,2021-01-12 06:45:00,15,24.0000,8,99.0000,36.9000,"18,639.2822",134.0000,63.0000,...,30.9000,96.5000,NaN,NaN,NaN,NaN,NaN,44.1538,113.7692,25.3077
1,47438179,2021-01-12 07:15:00,15,23.6667,8,104.6667,36.9000,"18,639.2984",115.3333,52.3333,...,30.9000,96.5000,NaN,NaN,NaN,NaN,NaN,44.1538,113.7692,25.3077
2,47438179,2021-01-12 07:45:00,15,23.7500,8,106.7500,36.5500,"18,639.3234",102.5000,50.5000,...,30.9000,96.5000,NaN,NaN,NaN,NaN,NaN,44.1538,113.7692,25.3077
3,47438179,2021-01-12 08:15:00,15,24.5000,8,108.2500,36.2250,"18,639.3474",100.5000,53.2500,...,30.9000,96.5000,NaN,NaN,NaN,NaN,NaN,44.1538,113.7692,25.3077
4,47438179,2021-01-12 08:45:00,15,25.5000,10,112.7500,36.2750,"18,639.3659",103.5000,51.7500,...,30.9000,96.5000,NaN,NaN,NaN,NaN,NaN,44.1538,113.7692,25.3077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189147,45308426,2020-09-11 15:30:00,15,18.0000,0,83.2857,36.6286,"18,516.6484",115.3333,71.9048,...,32.5000,94.0000,2.3500,0.9100,0.0500,0.3000,0.0300,33.0000,104.0000,7.0000
1189148,45308426,2020-09-11 16:00:00,15,18.0000,0,83.5714,36.6571,"18,516.6687",116.6667,71.8095,...,32.5000,94.0000,2.3500,0.9100,0.0500,0.3000,0.0300,33.0000,104.0000,7.0000
1189149,45308426,2020-09-11 16:30:00,15,18.0000,0,83.8571,36.6857,"18,516.6891",118.0000,71.7143,...,32.5000,94.0000,2.3500,0.9100,0.0500,0.3000,0.0300,33.0000,104.0000,7.0000
1189150,45308426,2020-09-11 17:00:00,15,18.0000,0,84.1429,36.7143,"18,516.7095",119.3333,71.6190,...,32.5000,94.0000,2.3500,0.9100,0.0500,0.3000,0.0300,33.0000,104.0000,7.0000


# Resumen

In [14]:
admin = 36937728
admin = 54960131

print("__________________________________________________________________________________________________________")
print("ADMISSION INFORMATION")
display(df_admissions[df_admissions[admission_field]==admin])

print("")
print("")
print("=========================== MIX time series BEFORE interpolation ====================================")
display(df[df[admission_field] == admin].iloc[:5])

print("")
print("")
print("=========================== MIX time series AFTER interpolation ====================================")
display(df_eobs_haem_mx[df_eobs_haem_mx[admission_field] == admin].iloc[:5])

print("__________________________________________________________________________________________________________")
print("")
print("Number of readings in timeseries BEFORE interpolation", len(df[df[admission_field] == admin]))
print("Number of readings in timeseries AFTER  interpolation", len(df_eobs_haem_mx[df_eobs_haem_mx[admission_field] == admin]))

__________________________________________________________________________________________________________
ADMISSION INFORMATION


,admission_id,episode_diagnoses,patient_id,age_at_admission\r,admission_date_time,admission_date_code_time,discharge_date_time,discharge_date_code_time,lengthofstay,isPneumonia,...,icu_count\r,Comorbidity_score,no_obs_eobs,no_haematology_eobs,receivedMedicines,Specific Comorbidity,had_Prev_admin,adm_has_spin,CURB65,antibiotic_4h
19005,54960131,"||C18.4,C77.2,Z50.1,J18.1,M13.9,F32.9,F41.9,D6...",17278697,58,2020-03-19 07:15:00,"18,340.3021",2020-03-28 19:00:00,"18,349.7917",9 days 11:45:00,1,...,0,6,53,8,0,0,1.0000,NaN,NaN,NaN




=========================== MIX time series BEFORE interpolation ====================================


,admission_id,timestamp,time_since_prev_obs_in_mins,rr,ews,heart_rate,temperature\r,timestamp_code,sbp,dbp,...,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI
1358208,54960131,2020-03-19 07:45:00,15,16.0000,1,77.0000,36.6000,"18,340.3274",109.0000,78.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1358209,54960131,2020-03-19 08:00:00,15,15.9900,1,76.7900,36.6030,"18,340.3377",108.9800,77.8600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1358210,54960131,2020-03-19 08:15:00,15,15.9800,1,76.5800,36.6060,"18,340.3481",108.9600,77.7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1358211,54960131,2020-03-19 08:30:00,15,15.9700,1,76.3700,36.6090,"18,340.3584",108.9400,77.5800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1358212,54960131,2020-03-19 08:45:00,15,15.9600,1,76.1600,36.6120,"18,340.3688",108.9200,77.4400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN




=========================== MIX time series AFTER interpolation ====================================


,admission_id,timestamp,time_since_prev_obs_in_mins,rr,ews,heart_rate,temperature\r,timestamp_code,sbp,dbp,...,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI
1358208,54960131,2020-03-19 07:45:00,15,16.0000,1,77.0000,36.6000,"18,340.3274",109.0000,78.0000,...,28.6000,89.0000,6.6900,0.8300,0.0300,0.1000,0.0100,42.0000,108.0000,8.0000
1358210,54960131,2020-03-19 08:15:00,15,15.9800,1,76.5800,36.6060,"18,340.3481",108.9600,77.7200,...,28.6000,89.0000,6.6900,0.8300,0.0300,0.1000,0.0100,42.0000,108.0000,8.0000
1358212,54960131,2020-03-19 08:45:00,15,15.9600,1,76.1600,36.6120,"18,340.3688",108.9200,77.4400,...,28.6000,89.0000,6.6900,0.8300,0.0300,0.1000,0.0100,42.0000,108.0000,8.0000
1358214,54960131,2020-03-19 09:15:00,15,15.9400,1,75.7400,36.6180,"18,340.3895",108.8800,77.1600,...,28.6000,89.0000,6.6900,0.8300,0.0300,0.1000,0.0100,42.0000,108.0000,8.0000
1358216,54960131,2020-03-19 09:45:00,15,15.9200,1,75.3200,36.6240,"18,340.4102",108.8400,76.8800,...,28.6000,89.0000,6.6900,0.8300,0.0300,0.1000,0.0100,42.0000,108.0000,8.0000


__________________________________________________________________________________________________________

Number of readings in timeseries BEFORE interpolation 288
Number of readings in timeseries AFTER  interpolation 144


# Add Confusion to data
The confusion field is a binary field computed from the Early warning score. 
Document NEWS2 Chart 1_The NEWS scoring system from [Link](https://www.rcplondon.ac.uk/projects/outputs/national-early-warning-score-news-2)

In [15]:
# -----------------------------------------------------------------
# Functions to find confusion
# -----------------------------------------------------------------
def ews_rr_score(rr_row):
    rr_score = 0
    if (rr_row >= 25) or (rr_row < 9):     rr_score = 3
    elif (21 <= rr_row) and (rr_row <= 24):rr_score = 2
    elif (9 <= rr_row) and (rr_row <= 11): rr_score = 1
    else: rr_score = 0
    return rr_score
# -----------------------------------------------------------------
# Oxygen saturation Score
def ews_ox_score(ox_row, as_ox_row):
    ox_score = 0
    if (ox_row <= 91): ox_score = 3
    elif (92 <= ox_row) and (ox_row <= 93): ox_score = 2
    elif (93 < ox_row) and (ox_row <= 95): ox_score  = 1
    else: ox_score = 0        
    if as_ox_row == 1: ox_score = ox_score + 2
    return ox_score

# -----------------------------------------------------------------
# Systolic blood presure score
def ews_sbp_score(sbp_row):
    sbp_score = 0
    if (sbp_row >= 220) or (sbp_row <= 90):    sbp_score = 3
    elif (91 <= sbp_row) and (sbp_row <= 100): sbp_score = 2
    elif (101 <= sbp_row) and (sbp_row <= 110):sbp_score = 1
    else: sbp_score = 0
    return sbp_score

# -----------------------------------------------------------------
# Heart rate score
def ews_hr_score(hr_row):
    hr_score = 0
    if (hr_row >= 131) or (hr_row <= 40):    hr_score = 3
    elif (111 <= hr_row) and (hr_row <= 130): hr_score = 2
    elif (91 <= hr_row) and (hr_row <= 110):hr_score = 1
    elif (41 <= hr_row) and (hr_row <= 50): hr_score = 1
    else: hr_score = 0
    return hr_score

# -----------------------------------------------------------------
# Temperature score
def ews_temp_score(temp_row):
    temp_score = 0
    if (temp_row <= 31.5):   temp_score = 3
    elif (39.1 <= temp_row): temp_score = 2
    elif (35.1 <= temp_row) and (temp_row <= 36):temp_score = 1
    elif (38.1 <= temp_row) and (temp_row <= 39): temp_score = 1
    else: temp_score = 0
    return temp_score

# -----------------------------------------------------------------
# Compute Confusion using the information of the Row
def confusion_from_row(row):
    confusion_row = 0
    ews_row = row['ews'] 
    rr_row = row['rr']
    ox_row    = row['Oxygen_Saturation']
    as_ox_row = row['Assisted_O2']
    sbp_row    = row['sbp']
    hr_row    = row['heart_rate']
    temp_row    = row['temperature\r']
    ews_score = ews_rr_score(rr_row)
    ews_score = ews_score + ews_ox_score(ox_row, as_ox_row)
    ews_score = ews_score + ews_sbp_score(sbp_row)
    ews_score = ews_score + ews_hr_score(hr_row)
    ews_score = ews_score + ews_temp_score(temp_row)
    if (abs(ews_score - ews_row) <= 3) and (abs(ews_score - ews_row) > 0): confusion_row = 1
    return confusion_row 

In [16]:
t = time.time()
df_eobs_haem_mx['Confusion'] = df_eobs_haem_mx.apply(lambda x:confusion_from_row(x), axis = 1)
col = df_eobs_haem_mx.pop('Confusion')
df_eobs_haem_mx.insert(12, col.name, col)

print("time elapsed: ", time.time() - t)

time elapsed:  34.967543840408325


In [17]:
df_eobs_haem_mx.columns

Index(['admission_id', 'timestamp', 'time_since_prev_obs_in_mins', 'rr', 'ews',
       'heart_rate', 'temperature\r', 'timestamp_code', 'sbp', 'dbp',
       'Oxygen_Saturation', 'Assisted_O2', 'Confusion', 'CREA', 'UREA', 'K',
       'GFR', 'WBC', 'PLT', 'HCT', 'HGB', 'RBC', 'MCH', 'MCV', 'NEUAB',
       'TLYMAB', 'EOSAB', 'MONAB', 'BASAB', 'ALB', 'ALP', 'BILI'],
      dtype='object')

In [18]:
df_eobs_haem_mx.head(5)

,admission_id,timestamp,time_since_prev_obs_in_mins,rr,ews,heart_rate,temperature\r,timestamp_code,sbp,dbp,...,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI
0,47438179,2021-01-12 06:45:00,15,24.0000,8,99.0000,36.9000,"18,639.2822",134.0000,63.0000,...,30.9000,96.5000,NaN,NaN,NaN,NaN,NaN,44.1538,113.7692,25.3077
2,47438179,2021-01-12 07:15:00,15,23.6667,8,104.6667,36.9000,"18,639.2984",115.3333,52.3333,...,30.9000,96.5000,NaN,NaN,NaN,NaN,NaN,44.1538,113.7692,25.3077
4,47438179,2021-01-12 07:45:00,15,23.7500,8,106.7500,36.5500,"18,639.3234",102.5000,50.5000,...,30.9000,96.5000,NaN,NaN,NaN,NaN,NaN,44.1538,113.7692,25.3077
6,47438179,2021-01-12 08:15:00,15,24.5000,8,108.2500,36.2250,"18,639.3474",100.5000,53.2500,...,30.9000,96.5000,NaN,NaN,NaN,NaN,NaN,44.1538,113.7692,25.3077
8,47438179,2021-01-12 08:45:00,15,25.5000,10,112.7500,36.2750,"18,639.3659",103.5000,51.7500,...,30.9000,96.5000,NaN,NaN,NaN,NaN,NaN,44.1538,113.7692,25.3077


# Cleanning the final mixed data frame

In [19]:
df_eobs_haem_mx.isna().sum()

admission_id                       0
timestamp                          0
time_since_prev_obs_in_mins        0
rr                                 0
ews                                0
heart_rate                         0
temperature\r                      0
timestamp_code                     0
sbp                                0
dbp                                0
Oxygen_Saturation                  0
Assisted_O2                        0
Confusion                          0
CREA                           13957
UREA                           13957
K                              13957
GFR                            14821
WBC                            13957
PLT                            14821
HCT                            14101
HGB                            13957
RBC                            14101
MCH                            14101
MCV                            14101
NEUAB                          26197
TLYMAB                         26341
EOSAB                          26341
M

In [20]:
t =time.time()
df_eobs_haem_mx_v2 = pd.DataFrame(columns = df_eobs_haem_mx.columns)
for adm in df_eobs_haem_mx[admission_field].unique().tolist():
    df_adm = df_eobs_haem_mx[df_eobs_haem_mx[admission_field] == adm].copy()
    if df_adm.isna().sum().sum() == 0:
        df_eobs_haem_mx_v2 = pd.concat([df_eobs_haem_mx_v2, df_adm])
print("elpased:", time.time()-t)   

elpased: 3514.8200948238373


In [21]:
df_eobs_haem_mx_v2.isna().sum()

admission_id                   0
timestamp                      0
time_since_prev_obs_in_mins    0
rr                             0
ews                            0
heart_rate                     0
temperature\r                  0
timestamp_code                 0
sbp                            0
dbp                            0
Oxygen_Saturation              0
Assisted_O2                    0
Confusion                      0
CREA                           0
UREA                           0
K                              0
GFR                            0
WBC                            0
PLT                            0
HCT                            0
HGB                            0
RBC                            0
MCH                            0
MCV                            0
NEUAB                          0
TLYMAB                         0
EOSAB                          0
MONAB                          0
BASAB                          0
ALB                            0
ALP       

In [22]:
df_eobs_haem_mx_v2.describe()

,admission_id,timestamp,time_since_prev_obs_in_mins,rr,ews,heart_rate,temperature\r,timestamp_code,sbp,dbp,...,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI
count,1136448,1136448,1136448,"1,136,448.0000",1136448,"1,136,448.0000","1,136,448.0000","1,136,448.0000","1,136,448.0000","1,136,448.0000",...,"1,136,448.0000","1,136,448.0000","1,136,448.0000","1,136,448.0000","1,136,448.0000","1,136,448.0000","1,136,448.0000","1,136,448.0000","1,136,448.0000","1,136,448.0000"
unique,7892,67454,1,"46,234.0000",18,"87,914.0000","70,472.0000","1,136,368.0000","97,422.0000","81,263.0000",...,"12,305.0000","4,403.0000","36,498.0000","17,549.0000","7,669.0000","11,404.0000","3,162.0000","3,838.0000","10,854.0000","4,896.0000"
top,70461223,2021-01-21 01:45:00,15,18.0000,2,90.0000,36.5000,"18,641.0157",111.0000,67.0000,...,29.5000,90.0000,15.0000,0.3000,0.0100,1.2000,0.0200,38.0000,200.0000,6.0000
freq,144,54,1136448,"204,527.0000",241247,"12,095.0000","20,678.0000",2.0000,"4,946.0000","7,778.0000",...,"7,606.0000","29,353.0000","63,164.0000","17,517.0000","84,401.0000","25,302.0000","110,649.0000","39,836.0000","72,702.0000","50,343.0000"


# Pickling Data

In [23]:
len(df_eobs_haem_mx_v2[admission_field].unique().tolist())

7892

In [24]:
df_eobs_haem_mx_v2.dtypes

admission_id                   object
timestamp                      object
time_since_prev_obs_in_mins    object
rr                             object
ews                            object
heart_rate                     object
temperature\r                  object
timestamp_code                 object
sbp                            object
dbp                            object
Oxygen_Saturation              object
Assisted_O2                    object
Confusion                      object
CREA                           object
UREA                           object
K                              object
GFR                            object
WBC                            object
PLT                            object
HCT                            object
HGB                            object
RBC                            object
MCH                            object
MCV                            object
NEUAB                          object
TLYMAB                         object
EOSAB       

In [25]:
features = df_eobs_haem_mx_v2.columns[2:]
for feat in features:
    if df_eobs_haem_mx_v2[feat].dtypes == 'O':
        df_eobs_haem_mx_v2[feat] = pd.to_numeric(df_eobs_haem_mx_v2[feat])
pickle.dump([df_eobs_haem_mx_v2], open('DataFrame_pickles/df_eobs_heam_mixed_2019_2020_V2.pickle', 'wb'))

In [26]:
t =time.time()
df_eobs_haem_mx_v3 = pd.DataFrame(columns = df_eobs_haem_mx.columns[:15])
for adm in df_eobs_haem_mx[admission_field].unique().tolist():
    df_adm = df_eobs_haem_mx[df_eobs_haem_mx[admission_field] == adm][df_eobs_haem_mx.columns[:15]].copy()
    if df_adm.isna().sum().sum() == 0:
        df_eobs_haem_mx_v3 = pd.concat([df_eobs_haem_mx_v3, df_adm])
print("elpased:", time.time()-t)   

elpased: 1956.0987224578857


In [27]:
len(df_eobs_haem_mx_v3[admission_field].unique().tolist())

8161

In [28]:
features = df_eobs_haem_mx_v3.columns[2:]
for feat in features:
    if df_eobs_haem_mx_v3[feat].dtypes == 'O':
        df_eobs_haem_mx_v3[feat] = pd.to_numeric(df_eobs_haem_mx_v3[feat])
pickle.dump([df_eobs_haem_mx_v3], open('DataFrame_pickles/df_eobs_heam_mixed_2019_2020_V3.pickle', 'wb'))